<h1 align="center">Find Enctyption Key Using Genetic Algorithm Artificial Intelligence</h1>

<h2 align="center">University of Tehran</h2>

<h6 align="center">Arash Mohamadpour 810197636</h6>

<h3> Introduction </h3>

In this assignment we have a ***global text*** and an ***encoded text*** and our goal is to find the ***encryption key*** using the **genetic algorithm**.
<br><br>

<h3> Gentic Algorithm </h3>

A genetic algorithm is a heuristic search method used in artificial intelligence and computing. It is used for ‍‍‍‍‍‍‍‍‍‍```finding optimized solutions``` to search problems based on the theory of natural selection and evolutionary biology. Genetic algorithms are excellent for searching through ```large and complex data sets```. They are considered capable of finding reasonable solutions to complex issues as they are highly capable of solving unconstrained and constrained optimization issues.

<h4> Population </h4>

Population is a ```subset of solutions``` in the current generation. It can also be defined as a set of chromosomes. There are several things to be kept in mind when dealing with GA population:
- The diversity of the population should be maintained otherwise it might lead to premature convergence.
- The population size should not be kept very large as it can cause a GA to slow down, while a smaller population might not be enough for a good mating pool. Therefore, an optimal population size needs to be decided by trial and error.

Here we're going to use ```DEFAULT_POPULATION_SIZE```, which is 100.

<h4> Chromosome </h4>

A chromosome is a set of parameters which define a proposed solution to the problem that the genetic algorithm is trying to solve.

<h4>Fitness Function</h4>

Fitness function evaluates how close a given solution is to the optimum solution of the desired problem. It determines how fit a solution is.

<h4>Selection</h4>

Selection is the stage of a genetic algorithm in which individual chromosomes are chosen from a population for later breeding.

In this assignment I'll use ```rank-based selection```.

<h5>Rank-based Selection</h5>
The population is sorted according to the objective values.
The fitness assigned to each individual depends only on its position in the individuals rank.
Ranking introduces a uniform scaling across the population and provides a simple and effective way of controlling selective pressure.

In [1]:
import string, time, re, random
import numpy as np
from operator import itemgetter

In [2]:
DEFAULT_POPULATION_SIZE = 150
ELITISM_RATE = 0.5
CROSSOVER_RATE = 0.9
SIZE_TWO = 2
alphabet_dict = {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25}


class Decoder:
    
    def __init__(self):
        self.encoded_text = self.extract_encoded_text();
        self.encoded_text = self.clean_text(self.encoded_text)
        self.encoded_words = self.encoded_text.split()
        
        self.create_dictionary()
    
    def extract_encoded_text(self):
        encoded_text = open("encoded_text.txt").read().lower()
        return encoded_text
    
    def create_dictionary(self):
        global_text = open("global_text.txt").read().lower()
        self.dictionary = set(self.clean_and_get_words(global_text))
    
    def clean_text(self, text):
        text = re.sub(r'[^A-Za-z]', ' ', text)
        words = text.split()
        clean_text = ' '
        for word in words:
            clean_text += word
            clean_text += ' '
        return clean_text
    
    def clean_and_get_words(self, text):
        cleaned_text = self.clean_text(text)
        words_set = cleaned_text.split()
        return words_set
        
    def create_random_chromosome(self):
        alphabet = list(string.ascii_lowercase)
        chromosome = ''
        for i in range(14):
            random_char = random.choice(alphabet)
            chromosome += random_char
        
        return chromosome
    
    def decrypt_char(self, enc_char, key):
        enc_char_index = string.ascii_lowercase.index(enc_char)
        key_index = string.ascii_lowercase.index(key)
        position_in_ascii_lowercase = enc_char_index - key_index
        if(position_in_ascii_lowercase >= 0):
            decrypted_char = self.get_alphabet_by_index(position_in_ascii_lowercase)
        else:
            decrypted_char = self.get_alphabet_by_index(26 + position_in_ascii_lowercase)
        
        return decrypted_char
    
    
    def decrypt(self, chromosome):
        decrypted_text = ''
        chromosome_pos = 0

        for i in range(len(self.encoded_text)):
            if (self.encoded_text[i] != ' '):
                pos = chromosome_pos % 14
                decrypted_text += self.decrypt_char(self.encoded_text[i], chromosome[pos])
                chromosome_pos += 1
            else:
                decrypted_text += ' '
        return decrypted_text
    
    def get_decrypted_words(self, chromosome):
        decrypted_text = self.decrypt(chromosome)
        decrypted_words = decrypted_text.split()
        return decrypted_words       
            
            
    
    def get_alphabet_by_index(self, position):
        for alphabet, index in alphabet_dict.items():
            if position == index:
                return alphabet
    
    
    def fitness(self, chromosome):
        decrypted_words = self.get_decrypted_words(chromosome)
        
        counter = 0
        for decrypted_word in decrypted_words:
            if decrypted_word in self.dictionary:
                counter += 1
                
        return counter
    
    def calculate_and_sort_population_fitness(self, population):
        population_scores = [[self.fitness(population[i]), i] for i in range(len(population))]
        sorted_population_scores = sorted(population_scores, key=itemgetter(0), reverse = True)
        return sorted_population_scores
    
    def two_point_crossover(self, mother, father):
        child = self.create_child(mother, father)
        return child
            
        
    def create_child(self, mother, father):
            point1 = random.randint(1, 13)
            child_chromosome = mother[:point1] + father[point1:]
            
            return child_chromosome
    
    
    def mutation(self, chromosome):
        chromosome = list(chromosome)
        for i in range(random.randint(0, 5)):
            point1 = random.randint(0, 13)
            point2 = random.randint(0, 13)
            
            temp = chromosome[point1]
            chromosome[point1] = chromosome[point2]
            chromosome[point2] = temp
            
        mutated_chromosome = "".join(chromosome)

        return mutated_chromosome
    
    
    def run_genetic_algorithm(self):
        words_len = len(self.encoded_words)
        population = [self.create_random_chromosome() for i in range(DEFAULT_POPULATION_SIZE)]
        
        sum_of_ranks = (len(population) + 1)/2
        ranks_probabilities = [(i+1)/sum_of_ranks for i in range(len(population))]
        ranks_probabilities = sorted(ranks_probabilities, reverse = True)
       
        while True:
            sorted_population_scores = self.calculate_and_sort_population_fitness(population)
            
            if sorted_population_scores[0][0] >= words_len:
                chromosome = population[sorted_population_scores[0][1]]
                print("The encryption key is: ")
                print(chromosome , "\n")
                
                return self.decrypt(chromosome)

          
            new_population = []
            new_population.extend([population[sorted_population_scores[i][1]] for i in range(int(DEFAULT_POPULATION_SIZE*ELITISM_RATE))])
            
            size = int(CROSSOVER_RATE*DEFAULT_POPULATION_SIZE) 
            
            for i in range(size):
                mother, father = np.random.choice(len(population), SIZE_TWO, ranks_probabilities)
                offspring = self.two_point_crossover(population[mother] , population[father]) 
                new_population.append(self.mutation(offspring))

            population = new_population
    

<br><br>

<h3>Running the test and decrypting the text</h3>

In [3]:
decoder = Decoder()

start = time.time()
decodedText = decoder.run_genetic_algorithm()
end = time.time()

print("Time: " + str(end-start), end="\n")

The encryption key is: 
alberteinstein 

Time: 80.45991826057434


In [4]:
print(decodedText)

 albert einstein old grove rd nassau point peconic long island august nd f d roosevelt president of the united states white house washington d c sir some recent work by e fermi and l szilard which has been communicated to me in manuscript leads me to expect that the element uranium may be turned into a new and important source of energy in the immediate future certain aspects of the situation which has arisen seem to call for watchfulness and if necessary quick action on the part of the administration i believe therefore that it is my duty to bring to your attention the following facts and recommendations in the course of the last four months it has been made probable through the work of joliot in france as well as fermi and szilard in america that it may become possible to set up a nuclear chain reaction in a large mass of uranium by which vast amounts of power and large quantities of new radium like elements would be generated now it appears almost certain that this could be achieved